In [1]:
# import necessary libraries
import json
from pathlib import Path
from langchain.agents import create_json_agent
from langchain_community.agent_toolkits import JsonToolkit
from langchain_community.llms.openai import OpenAI
from langchain_community.tools.json.tool import JsonSpec
from pprint import pprint
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
import ast

# 1.READ THE JSON FILE

In [2]:
# read json file
file_path="restaurant_menu.json"
data=json.loads(Path(file_path).read_text())
pprint(data)

{'breakfast': [{'availability': '7am - 11am',
                'description': 'Poached eggs on toasted English muffins with '
                               'Canadian bacon and hollandaise sauce.',
                'ingredients': ['Eggs',
                                'English muffin',
                                'Canadian bacon',
                                'Hollandaise sauce'],
                'name': 'Classic Eggs Benedict',
                'options': ['Gluten-free muffin', 'Turkey bacon'],
                'price': 12.99},
               {'availability': '7am - 11am',
                'description': 'Fluffy pancakes filled with fresh blueberries, '
                               'served with maple syrup and whipped cream.',
                'ingredients': ['Flour',
                                'Blueberries',
                                'Eggs',
                                'Milk',
                                'Maple syrup',
                                'Whipped 

In [3]:
openai_api_key=""

# 2.CREATE JSON AGENT EXCUTOR

In [4]:
with open("restaurant_menu.json",'r') as f:
    data = json.load(f)
json_spec = JsonSpec(dict_=data, max_value_length=4000)
json_toolkit = JsonToolkit(spec=json_spec)

json_agent_executor = create_json_agent(
    llm=OpenAI(openai_api_key=openai_api_key,temperature=0), toolkit=json_toolkit, verbose=False
)

In [5]:
json_agent_executor.run("what are the available coffee you have")

"[{'name': 'Espresso', 'description': 'Rich and bold espresso, freshly brewed.', 'price': 3.0, 'ingredients': ['Espresso'], 'options': ['Double shot', 'With milk'], 'availability': 'All day'}, {'name': 'Cappuccino', 'description': 'Espresso with steamed milk and a layer of frothy foam.', 'price': 4.5, 'ingredients': ['Espresso', 'Milk', 'Foam'], 'options': ['Vanilla flavor', 'Cinnamon'], 'availability': 'All day'}]"

# 3. CREATED MENU CHAIN TO RETURN ITEM NAME & PRICE 

In [6]:
# create a chatbot that connect with your json_agent_executer

from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

In [7]:
# Create a function to connect with json agent
def check__menu(type_prompt: str) -> str:
    response=json_agent_executor.run(type_prompt)
    return response
runnable_1 = RunnableLambda(check__menu)

parser=StrOutputParser()

In [8]:
menu_chain= (
    PromptTemplate.from_template(
        """List all the available {item} . return name and their prices available in the menu"""
    )
    | runnable_1|parser
)

In [9]:
json_agent_executor.run("what are the available coffee you have")

"[{'name': 'Espresso', 'description': 'Rich and bold espresso, freshly brewed.', 'price': 3.0, 'ingredients': ['Espresso'], 'options': ['Double shot', 'With milk'], 'availability': 'All day'}, {'name': 'Cappuccino', 'description': 'Espresso with steamed milk and a layer of frothy foam.', 'price': 4.5, 'ingredients': ['Espresso', 'Milk', 'Foam'], 'options': ['Vanilla flavor', 'Cinnamon'], 'availability': 'All day'}]"

In [10]:
menu_chain.invoke({'item':'coffee'})

'Espresso - $3.0, Cappuccino - $4.5'

# 4. CREATED INGREDIANT CHAIN TO RETURN THE PRESENCE

In [11]:
runnable_ingredient = RunnableLambda(check__menu)

ingrediants_chain=(PromptTemplate.from_template(
        """check if the {ingredient_to_check} is present in {food_item}. respond ingredient is present or ingredient is adsent to user"""
    )
    | runnable_ingredient|parser
)

In [12]:
ingrediants_chain.invoke({'food_item':'Vegetarian Lasagna','ingredient_to_check':'Ricotta'})

'ingredient is present'

# 5. CREATED A CLASSIFICATION CHAIN TO CLASSIFY THE REQUEST

In [13]:
runnable_general = RunnableLambda(check__menu)
chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as being about 'breakfast','coffee','main_course','starters','drinks',`ingrediants`,'other'.

Do not respond with more than one word.

{question}

Classification:"""
    )
    | runnable_general|StrOutputParser()
)

# 6. CREATED A GENERAL CHAIN TO RESPOND TO USER QUESTION

In [14]:
#general chain
runnable_general = RunnableLambda(check__menu)
general_chain=(PromptTemplate.from_template(
        """You have restaurent menu, Respond to user question {question}."""
    )
    |runnable_general|parser
)

# 7. CREATED A RUNNABLE BRANCH TO ROUTE TO DIFFERENT CHAIN

In [15]:
from langchain_core.runnables import RunnableBranch


lambda x: any(keyword in x["topic"].lower() for keyword in ["breakfast", "drink"])

branch = RunnableBranch(
    (lambda x: any(val in x["topic"].lower() for val in ['breakfast','coffee','main_course','starters','drinks']), menu_chain),
    (lambda x: "ingrediants" in x["topic"].lower(), ingrediants_chain),
    general_chain
)

In [16]:
full_chain = {"topic": chain, "question": lambda x: x["question"],"item":lambda x: x["item"] } | branch

In [17]:
#full_chain.invoke({"question": "what are the names of breakfast?"})

In [18]:
full_chain.invoke({
    "topic": "",
    "item": "breakfast",
    "question": "what are the names of breakfast?"
})

"[{'name': 'Classic Eggs Benedict', 'price': 12.99}, {'name': 'Blueberry Pancakes', 'price': 10.5}]"

In [19]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

In [25]:
full_chain.invoke({
    "topic": "",
    "food_item": "Classic Eggs Benedict",
    "ingredient_to_check": "Eggs",
    "question": "is Classic Eggs Benedict have ingredient Eggs?"
})

'Yes, Classic Eggs Benedict has the ingredient Eggs.'

In [30]:
full_chain.invoke({
    "topic": "",
    "food_item": "Classic Eggs Benedict",
    "ingredient_to_check": "Eggs",
    "question": "is Classic Eggs Benedict have ingredient Eggs?"
})

'Yes, Classic Eggs Benedict has the ingredient Eggs.'